<h1 style="color:darkblue; text-align:center; font-family:Arial, Helvetica, sans-serif; font-size:48px; font-weight:bold; text-shadow: 2px 2px 4px gray;">
Threat vs Hope — NASA Space Data Story
</h1>

<p style="color:black; font-size:16px; font-family:Georgia; text-align:justify;">
Space does not send a single, simple message. One side of the universe throws rocks past Earth, some of them skimming close enough to remind us how small and fragile our world is. The other side quietly hides distant planets that might one day feel like a second home, orbiting faraway stars we can barely see.
</p>

<p style="color:black; font-size:16px; font-family:Georgia; text-align:justify;">
In this notebook, both sides are introduced: the first part shows Near-Earth Objects (NEOs), highlighting how many dangerous rocks are out there, how large they are, and how close they come to Earth. The second part explores Exoplanets (Hope), where each row represents a distant planet with radius, temperature, and other properties that hint at how Earth-like it might be.
</p>


![Alt Text](NASA.png)


<h2 style="color:darkblue; font-family:Verdana;">
Story Objectives
</h2>
<hr style="border:1px solid darkblue;"/>

<p style="font-family:Georgia; text-align:justify;">
<b>Two datasets, two faces of space:</b>
</p>
<ul style="font-family:Georgia;">
<li><b>NEO (Threat):</b> Each row is an asteroid that approached Earth, with size, close-approach distance, and a hazardous flag. Together, they map the background level of danger in our cosmic neighborhood.</li>
<li><b>Exoplanets (Hope):</b> Each row is a candidate planet with radius, temperature, and other properties that hint at how Earth-like it might be.</li>
</ul>

<p style="font-family:Georgia; text-align:justify;">
After introducing both datasets, this notebook focuses on exploring and understanding their key features. The main objectives are:
</p>
<ul style="font-family:Georgia;">
<li>Perform an initial exploration of each dataset: examine shapes, data types, missing values, and basic distributions.</li>
<li>Clean and standardize the data to ensure numeric columns, proper types, and handle missing or inconsistent values.</li>
<li>Create derived metrics or scores that highlight important characteristics (e.g., <code>threat_score</code> for NEOs, habitability indicators for exoplanets).</li>
<li>Summarize and group data by relevant categories to extract meaningful patterns or trends.</li>
<li>Visualize key distributions, relationships, and comparisons with clear titles, labels, and color coding.</li>
<li>Highlight insights that tell a clear story about potential dangers from NEOs and promising features of exoplanets.</li>
</ul>


<h2 style="color:darkblue; font-family:Verdana;">
0. Setup
</h2>
<hr style="border:1px solid darkblue;"/>

<p style="font-family:Georgia;">
Import the required libraries and load the datasets.
</p>


In [1]:
# ============================
# Standard Libraries
# ============================
import json
import math
import requests
from datetime import datetime
from io import StringIO
from itertools import cycle
from pathlib import Path

import numpy as np
import pandas as pd
import warnings

# ============================
# Visualization Libraries
# ============================
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle, FancyBboxPatch
import seaborn as sns

# ============================
# Statistical Tools
# ============================
from scipy import stats

# ============================
# Machine Learning (Scikit-Learn)
# ============================
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_curve, auc,
    precision_recall_curve
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize

# ============================
# Warnings
# ============================
warnings.filterwarnings('ignore')

# ============================
# Display & Style Configuration
# ============================
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
sns.set(style="whitegrid")


In [2]:
# Load datasets (adjust paths if needed)
neo = pd.read_csv("datasets/neo.csv")
#exo = pd.read_csv("datasets/nasa-exoplanet-archive.csv")  

neo.head()

,Unnamed: 0,id,neo_reference_id,name,name_limited,designation,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,kilometers_estimated_diameter_min,kilometers_estimated_diameter_max,orbit_id,orbit_class_type,perihelion_distance,aphelion_distance,first_observation_date,last_observation_date,orbit_class_description
0,0,2001981,2001981,1981 Midas (1973 EA),Midas,1981,15.220,True,False,2.402,5.371,229,APO,0.622,2.931,1973-03-06,2021-10-20,Near-Earth asteroid orbits which cross the Ear...
1,1,2002059,2002059,2059 Baboquivari (1963 UA),Baboquivari,2059,15.970,False,False,1.700,3.802,268,AMO,1.239,4.048,1963-10-16,2021-04-15,Near-Earth asteroid orbits similar to that of ...
2,2,2002061,2002061,2061 Anza (1960 UA),Anza,2061,16.360,False,False,1.421,3.177,180,AMO,1.050,3.478,1960-10-22,2021-06-12,Near-Earth asteroid orbits similar to that of ...
3,3,2002062,2002062,2062 Aten (1976 AA),Aten,2062,17.100,False,False,1.011,2.260,149,ATE,0.790,1.144,1955-12-17,2019-11-10,Near-Earth asteroid orbits similar to that of ...
4,4,2002063,2002063,2063 Bacchus (1977 HB),Bacchus,2063,17.280,False,False,0.930,2.080,143,APO,0.701,1.455,1977-04-24,2021-12-02,Near-Earth asteroid orbits which cross the Ear...


<h1 style="color:crimson; text-align:center; font-family:Verdana; font-size:36px; line-height:1.2;">
⚠️ Near-Earth Rocks: Tracking Earth's Cosmic Visitors ⚠️
</h1>


![Alt Text](NEO.png)

<h2 style="color:darkblue; font-family:Verdana;">
1. EDA — Exploring the NEO Dataset
</h2>
<hr style="border:1px solid darkblue;"/>

<p style="font-family:Georgia;">
Let's explore the NEO dataset structure, data types, and basic statistics before defining any threat metrics.
</p>

In [3]:
# Display basic information about the NEO dataset
print("=" * 60)
print("DATASET SHAPE (NEO)")
print("=" * 60)
print(f"Rows: {neo.shape[0]:,}")
print(f"Columns: {neo.shape[1]}")
print()

print("=" * 60)
print("FIRST FEW ROWS")
print("=" * 60)
neo.head(10)

DATASET SHAPE (NEO)
Rows: 24,000
Columns: 18

FIRST FEW ROWS


,Unnamed: 0,id,neo_reference_id,name,name_limited,designation,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,kilometers_estimated_diameter_min,kilometers_estimated_diameter_max,orbit_id,orbit_class_type,perihelion_distance,aphelion_distance,first_observation_date,last_observation_date,orbit_class_description
0,0,2001981,2001981,1981 Midas (1973 EA),Midas,1981,15.220,True,False,2.402,5.371,229,APO,0.622,2.931,1973-03-06,2021-10-20,Near-Earth asteroid orbits which cross the Ear...
1,1,2002059,2002059,2059 Baboquivari (1963 UA),Baboquivari,2059,15.970,False,False,1.700,3.802,268,AMO,1.239,4.048,1963-10-16,2021-04-15,Near-Earth asteroid orbits similar to that of ...
2,2,2002061,2002061,2061 Anza (1960 UA),Anza,2061,16.360,False,False,1.421,3.177,180,AMO,1.050,3.478,1960-10-22,2021-06-12,Near-Earth asteroid orbits similar to that of ...
3,3,2002062,2002062,2062 Aten (1976 AA),Aten,2062,17.100,False,False,1.011,2.260,149,ATE,0.790,1.144,1955-12-17,2019-11-10,Near-Earth asteroid orbits similar to that of ...
4,4,2002063,2002063,2063 Bacchus (1977 HB),Bacchus,2063,17.280,False,False,0.930,2.080,143,APO,0.701,1.455,1977-04-24,2021-12-02,Near-Earth asteroid orbits which cross the Ear...
5,5,2002100,2002100,2100 Ra-Shalom (1978 RA),Ra-Shalom,2100,16.230,False,False,1.509,3.373,278,ATE,0.469,1.195,1975-10-03,2021-12-02,Near-Earth asteroid orbits similar to that of ...
6,6,2002101,2002101,2101 Adonis (1936 CA),Adonis,2101,18.640,True,False,0.497,1.112,45,APO,0.442,3.306,1936-02-21,2020-06-09,Near-Earth asteroid orbits which cross the Ear...
7,7,2002102,2002102,2102 Tantalus (1975 YA),Tantalus,2102,16.000,True,False,1.677,3.750,222,APO,0.904,1.676,1975-12-30,2021-09-22,Near-Earth asteroid orbits which cross the Ear...
8,8,2002135,2002135,2135 Aristaeus (1977 HA),Aristaeus,2135,18.020,True,False,0.662,1.479,24,APO,0.795,2.404,1977-04-17,2020-05-30,Near-Earth asteroid orbits which cross the Ear...
9,9,2002201,2002201,2201 Oljato (1947 XC),Oljato,2201,15.250,True,False,2.369,5.297,234,APO,0.624,3.725,1931-12-03,2021-09-16,Near-Earth asteroid orbits which cross the Ear...


In [4]:
# Data types and basic info
print("=" * 60)
print("DATA TYPES AND INFO (NEO)")
print("=" * 60)
print(neo.dtypes)
print()
print(neo.info())

DATA TYPES AND INFO (NEO)
Unnamed: 0                             int64
id                                     int64
neo_reference_id                       int64
name                                  object
name_limited                          object
designation                           object
absolute_magnitude_h                 float64
is_potentially_hazardous_asteroid       bool
is_sentry_object                        bool
kilometers_estimated_diameter_min    float64
kilometers_estimated_diameter_max    float64
orbit_id                              object
orbit_class_type                      object
perihelion_distance                  float64
aphelion_distance                    float64
first_observation_date                object
last_observation_date                 object
orbit_class_description               object
dtype: object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 18 columns):
 #   Column                             

In [5]:
# Check for missing values
print("=" * 60)
print("MISSING VALUES (NEO)")
print("=" * 60)
missing = neo.isnull().sum()
missing_pct = (missing / len(neo)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing Percentage': missing_pct
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
if len(missing_df) > 0:
    print(missing_df)
else:
    print("No missing values found in NEO dataset!")

MISSING VALUES (NEO)
                                   Missing Count  Missing Percentage
name_limited                               23856              99.400
absolute_magnitude_h                           8               0.033
kilometers_estimated_diameter_min              8               0.033
kilometers_estimated_diameter_max              8               0.033


In [6]:
# Basic statistics for numerical columns
print("=" * 60)
print("NUMERICAL SUMMARY STATISTICS (NEO)")
print("=" * 60)
neo.describe().T

NUMERICAL SUMMARY STATISTICS (NEO)


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,24000.000,11999.500,6928.348,0.000,5999.750,11999.500,17999.250,23999.000
id,24000.000,7275110.744,13339866.241,2001981.000,3409961.750,3703790.500,3803896.750,54087490.000
neo_reference_id,24000.000,7275110.744,13339866.241,2001981.000,3409961.750,3703790.500,3803896.750,54087490.000
absolute_magnitude_h,23992.000,22.942,2.953,12.580,20.650,23.200,25.200,33.200
kilometers_estimated_diameter_min,23992.000,0.168,0.293,0.001,0.024,0.061,0.197,8.101
kilometers_estimated_diameter_max,23992.000,0.375,0.655,0.001,0.054,0.136,0.441,18.115
perihelion_distance,24000.000,0.915,0.232,0.070,0.785,0.965,1.069,1.300
aphelion_distance,24000.000,2.653,4.468,0.654,1.706,2.480,3.398,631.895


<p style="font-family:Georgia; text-align:justify;">
This quick EDA reveals how many close approaches are recorded, which columns are reliable, and the overall scale of asteroid sizes and distances. It sets the stage for defining a threat score and focusing on the most concerning objects.
</p>


<h2 style="color:darkblue; font-family:Verdana;">
2. Data Cleaning
</h2>
<hr style="border:1px solid darkblue;"/>

<p style="font-family:Georgia;">
Clean and prepare the NEO dataset by converting key columns to numeric, filling missing values, standardizing hazard flags, and keeping extreme outliers for analysis of the most threatening asteroids.</p>


In [7]:
# Create a copy for cleaning
neo_clean = neo.copy()

print("Starting NEO data cleaning process...")
print(f"Original shape: {neo_clean.shape}")

Starting NEO data cleaning process...
Original shape: (24000, 18)


In [8]:
# Check and remove duplicate rows
print("=" * 60)
print("DUPLICATE ROWS (NEO - CLEAN)")
print("=" * 60)

dup_count = neo_clean.duplicated().sum()
print(f"Duplicate rows: {dup_count:,}")

if dup_count > 0:
    neo_clean = neo_clean.drop_duplicates()
    print(f"Removed {dup_count:,} duplicates")
    print(f"New shape: {neo_clean.shape}")

print()


DUPLICATE ROWS (NEO - CLEAN)
Duplicate rows: 0



In [9]:
# ----------------------------
# Unique values check
# ----------------------------
print("\n" + "="*60)
print("UNIQUE VALUES CHECK (NEO CLEAN DATA)")
print("="*60 + "\n")

for col in ["id", "name", "orbiting_body", "hazardous", "size_category"]:
    if col in neo_clean.columns:
        unique_vals = neo_clean[col].nunique()
        print(f"{col}: {unique_vals:,} unique values")
        # opsionale: shfaq 5 të parët
        print(f"   Example values: {neo_clean[col].unique()[:5]}\n")

print("="*70 + "\n")



UNIQUE VALUES CHECK (NEO CLEAN DATA)

id: 24,000 unique values
   Example values: [2001981 2002059 2002061 2002062 2002063]

name: 24,000 unique values
   Example values: ['1981 Midas (1973 EA)' '2059 Baboquivari (1963 UA)' '2061 Anza (1960 UA)'
 '2062 Aten (1976 AA)' '2063 Bacchus (1977 HB)']




In [10]:
# Ensure numeric types for key columns
numeric_cols = ["estimated_diameter_max", "miss_distance_km", "relative_velocity_km_s"]
for col in numeric_cols:
    if col in neo_clean.columns:
        neo_clean[col] = pd.to_numeric(neo_clean[col], errors="coerce")

# Handle missing values in threat-related columns
for col in ["estimated_diameter_max", "miss_distance_km"]:
    if col in neo_clean.columns:
        if neo_clean[col].isnull().sum() > 0:
            print(f"Filling missing values in {col} with median")
            neo_clean[col] = neo_clean[col].fillna(neo_clean[col].median())

# Convert hazard flag to boolean/category
if "is_potentially_hazardous_asteroid" in neo_clean.columns:
    neo_clean["is_potentially_hazardous_asteroid"] = neo_clean[
        "is_potentially_hazardous_asteroid"
    ].astype("bool")
    print("Converted is_potentially_hazardous_asteroid to bool")

print("\nData types after cleaning:")
print(neo_clean.dtypes.head(15))

Converted is_potentially_hazardous_asteroid to bool

Data types after cleaning:
Unnamed: 0                             int64
id                                     int64
neo_reference_id                       int64
name                                  object
name_limited                          object
designation                           object
absolute_magnitude_h                 float64
is_potentially_hazardous_asteroid       bool
is_sentry_object                        bool
kilometers_estimated_diameter_min    float64
kilometers_estimated_diameter_max    float64
orbit_id                              object
orbit_class_type                      object
perihelion_distance                  float64
aphelion_distance                    float64
dtype: object


In [11]:
# ----------------------------
# Outlier check with IQR method
# ----------------------------
print("\n" + "="*60)
print("OUTLIER CHECK (IQR METHOD)")
print("="*60 + "\n")

# List of numeric columns to check for outliers
num_cols = [
    "estdiametermin",
    "estdiametermax",
    "relativevelocity",
    "missdistance",
    "absolutemagnitude"
]

outlier_info = []

for col in num_cols:
    if col in neo_clean.columns:
        Q1 = neo_clean[col].quantile(0.25)
        Q3 = neo_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        mask = (neo_clean[col] < lower) | (neo_clean[col] > upper)
        count = mask.sum()
        pct = (count / len(neo_clean)) * 100
        outlier_info.append((col, count, pct))
        print(f"{col}: {count:,} outliers ({pct:.2f}%)")

print("\nNote: Outliers are kept in the dataset because extreme values")
print("can represent the most interesting NEOs (very large or very close).")
print(f"\nFINAL CLEAN DATA SHAPE: {neo_clean.shape}\n")



OUTLIER CHECK (IQR METHOD)


Note: Outliers are kept in the dataset because extreme values
can represent the most interesting NEOs (very large or very close).

FINAL CLEAN DATA SHAPE: (24000, 18)



<h2 style="color:darkblue; font-family:Verdana;">
2. Data Transformation 
</h2>
<hr style="border:1px solid darkblue;"/>

<p style="font-family:Georgia;">
Transform the cleaned NEO dataset by creating a <code>threat_score</code>, categorizing asteroids by size and distance, flagging big and/or close hazards, and summarizing potentially dangerous objects.</p>


In [12]:
# ----------------------------
# Threat Score and Hazardous Asteroids
# ----------------------------

# Convert 'miss_distance' to float to ensure numeric operations
neo_clean["miss_distance"] = neo_clean["miss_distance"].astype(float)

# Calculate 'threat_score' as asteroid size divided by miss distance
# Higher values indicate potentially more dangerous objects
neo_clean["threat_score"] = neo_clean["est_diameter_max"] / neo_clean["miss_distance"]

# Filter only the potentially hazardous asteroids
# 'hazardous' column is assumed to be boolean (True/False)
dangerous_asteroids = neo_clean[neo_clean["hazardous"] == True]

# Display a sample of the dangerous asteroids
print("=" * 60)
print("DANGEROUS ASTEROIDS (SAMPLE)")
print("=" * 60)
dangerous_asteroids.head(10)  # Show first 10 rows for quick inspection


KeyError: 'miss_distance'

<p style="font-family:Georgia; text-align:justify;">
Each asteroid now carries a <code>threat_score</code> that rises when the object is both large and close. Filtering to rows marked as potentially hazardous gives the core cast of dangerous NEOs that will drive the visual story.
</p>


In [ ]:
# ----------------------------
# Categorize Asteroids by Size 
# ----------------------------
print("SIZE CATEGORIES (based on max diameter)")

# Pick the right diameter column name
if "est_diameter_max" in neo_clean.columns:
    size_col = "est_diameter_max"
else:
    size_col = "estdiametermax"

# Define bins (7 edges → 6 intervals)
size_bins = [0, 0.05, 0.1, 0.5, 1, 5, 50]  # km

# Define 6 labels for 6 intervals
size_labels = ["Tiny", "Very Small", "Small", "Medium", "Large", "Very Large"]

# Categorize each asteroid by size
neo_clean["size_category"] = pd.cut(
    neo_clean[size_col],
    bins=size_bins,
    labels=size_labels,
    right=False
)

# Display the counts of asteroids in each size category
print(neo_clean["size_category"].value_counts().sort_index())
print()

In [ ]:
# ----------------------------
# Categorize Asteroids by Miss Distance (Distance Risk)
# ----------------------------
print("DISTANCE RISK CATEGORIES (based on miss distance)")

# Determine the correct distance column
if "miss_distance" in neo_clean.columns:
    dist_col = "miss_distance"
else:
    dist_col = "missdistance"

# Define distance bins in km (edges)
# 5 edges → 4 intervals, so we need 4 labels
dist_bins = [0, 1e7, 3e7, 5e7, 1e8]  # e.g., 10M, 30M, 50M, 100M km

# Labels for each distance interval (must be 4 labels for 4 intervals)
dist_labels = ["Extremely Close", "Very Close", "Close", "Moderate"]

# Categorize asteroids by miss distance
neo_clean["distance_risk"] = pd.cut(
    neo_clean[dist_col],
    bins=dist_bins,
    labels=dist_labels,
    right=False
)

# Display counts for each distance risk category
print(neo_clean["distance_risk"].value_counts().sort_index())
print()


In [ ]:
# ----------------------------
# Derived Threat Flags
# ----------------------------
print("DERIVED THREAT FLAGS")

# Column indicating whether an asteroid is potentially hazardous
haz_col = "hazardous"  # adjust if your column name differs

# ----------------------------
# "Big" threat: diameter > 0.5 km AND hazardous
# ----------------------------
neo_clean["is_big_threat"] = (neo_clean[size_col] > 0.5) & (neo_clean[haz_col])

# ----------------------------
# "Close" threat: miss distance < 30 million km AND hazardous
# ----------------------------
neo_clean["is_close_threat"] = (neo_clean[dist_col] < 3e7) & (neo_clean[haz_col])

# ----------------------------
# "Double" threat: both big AND close
# ----------------------------
neo_clean["is_double_threat"] = neo_clean["is_big_threat"] & neo_clean["is_close_threat"]

# ----------------------------
# Display counts for each threat type
# ----------------------------
print(f"Big threats: {neo_clean['is_big_threat'].sum():,}")
print(f"Close threats: {neo_clean['is_close_threat'].sum():,}")
print(f"Double threats: {neo_clean['is_double_threat'].sum():,}")
print()


In [ ]:
# ----------------------------
# Display Hazardous Asteroids Sorted by Threat Score
# ----------------------------
print("HAZARDOUS ASTEROIDS (SAMPLE BY THREAT SCORE)")

# Filter only hazardous asteroids and sort by threat_score descending
dangerous_asteroids = (
    neo_clean[neo_clean[haz_col] == True]  # keep only hazardous asteroids
    .sort_values("threat_score", ascending=False)  # sort highest threat first
)

# Print total number of hazardous NEOs
print(f"Number of hazardous NEOs: {len(dangerous_asteroids):,}")
print()

# Display top 10 hazardous asteroids by threat_score
dangerous_asteroids.head(10)


In [ ]:
# ----------------------------
# Summary by Hazard Flag and Size Category
# ----------------------------
print("SUMMARY BY HAZARD FLAG AND SIZE CATEGORY")

# Group the dataset by:
# 1. Hazardous flag (haz_col) → True/False
# 2. Size category → Tiny, Very Small, Small, etc.
summary = (
    neo_clean
    .groupby([haz_col, "size_category"], observed=True)  # observed=True avoids including unused categories
    .agg(
        # Count of asteroids in each group
        # If 'id' exists, count by 'id', else use 'threat_score'
        count=("id", "count") if "id" in neo_clean.columns else ("threat_score", "count"),
        # Average threat score per group
        mean_threat=("threat_score", "mean"),
        # Maximum threat score per group
        max_threat=("threat_score", "max")
    )
    .round(6)  # round numeric results to 6 decimal places for readability
)

# Print the grouped summary
print(summary)

# Print final number of columns in the cleaned dataset
print("\nFINAL NUMBER OF COLUMNS IN neo_clean:", len(neo_clean.columns))


<h2 style="color:darkblue; font-family:Verdana;">
4. Visualizing the Threat
</h2>
<hr style="border:1px solid darkblue;"/>

<p style="font-family:Georgia;">
Now we turn the transformed NEO data into graphs that show how many asteroids exist in each size category and what fraction of them are hazardous.</p>


In [ ]:
# ----------------------------
# Distribution of asteroid diameters (log scale)
# ----------------------------
plt.figure(figsize=(10, 6))
sns.histplot(
    data=neo_clean,
    x="est_diameter_max",  # maximum estimated diameter of asteroid
    bins=30,
    kde=True,
    log_scale=True        # log scale for better visualization
)
plt.title("Distribution of Asteroid Diameters (Log Scale)", fontsize=14)
plt.xlabel("Estimated Diameter (km, log scale)", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.tight_layout()
plt.show()

<p style="font-family:Georgia; text-align:justify;">
This histogram shows the distribution of maximum estimated asteroid diameters on a logarithmic scale: larger values on the x‑axis mean bigger asteroids, and higher bars mean that size range is more common in the dataset.
</p>


In [ ]:
# ----------------------------
# Count of hazardous vs non-hazardous NEOs by orbiting body
# ----------------------------
plt.figure(figsize=(7, 4))
sns.countplot(
    data=neo_clean,
    x="orbiting_body",
    hue="hazardous",
    palette={False: "skyblue", True: "salmon"} 
)
plt.title("Count of Hazardous vs Non-Hazardous NEOs by Orbiting Body")
plt.xlabel("Orbiting Body")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


<p style="font-family:Georgia; text-align:justify;">
This bar chart compares the number of hazardous and non-hazardous Near-Earth Objects (NEOs) that orbit Earth. It shows that most NEOs are non-hazardous, while only a much smaller portion is classified as hazardous.
</p>


In [ ]:
# ----------------------------
# Scatter: asteroid size vs miss distance (hazardous only)
# ----------------------------
plt.figure(figsize=(8, 5))

# Scatterplot për vetëm asteroidet e rrezikshme
sns.scatterplot(
    data=neo[neo['hazardous'] == True],  # filter inline
    x="est_diameter_max",
    y="miss_distance"
)

plt.title("Asteroid Size vs Miss Distance (Hazardous NEOs)")
plt.xlabel("Estimated Diameter (km)")
plt.ylabel("Miss Distance (km)")
plt.tight_layout()
plt.show()


<p style="font-family:Georgia; text-align:justify;">
Each point in this scatter plot is a potentially hazardous asteroid. This scatter plot shows how the estimated diameter of potentially hazardous asteroids relates to their miss distance from Earth: the bigger the asteroid, the more important it is to check whether it comes close to Earth.
</p>


In [ ]:
# ----------------------------
# Relative velocity distribution by hazard flag
# ----------------------------
# Ensure relative_velocity is numeric
neo_clean["relative_velocity"] = neo_clean["relative_velocity"].astype(float)

fig, ax = plt.subplots(figsize=(8, 5))
sns.kdeplot(
    data=neo_clean,
    x="relative_velocity",
    hue="hazardous",
    common_norm=False,                # density is not normalized together
    palette={False: "skyblue", True: "salmon"},
    ax=ax
)
ax.set_title("Relative Velocity by Hazardous Flag")
ax.set_xlabel("Relative Velocity (km/s)")
ax.set_ylabel("Density")
plt.tight_layout()
plt.show()

<p style="font-family:Georgia; text-align:justify;">
This plot compares the distribution of relative velocities for hazardous and non‑hazardous NEOs: both groups peak around tens of thousands of km/s, but hazardous asteroids (red line) tend to have slightly higher typical velocities and a longer high‑speed tail than non‑hazardous ones (blue line).
The higher the density of an asteroid, the more mass it has in the same size, so if it is moving fast it hits harder and causes more damage.
</p>


In [ ]:
# ----------------------------
# Boxplot: Miss Distance in Lunar Distance vs Hazardous Flag
# ----------------------------
# Convert miss distance to Lunar Distance
neo_clean["miss_distance_ld"] = neo_clean["miss_distance"] / 384400  # 1 LD = 384,400 km

fig, ax = plt.subplots(figsize=(8, 5))
sns.boxplot(
    data=neo_clean,
    x="hazardous",
    y="miss_distance_ld",
    hue="hazardous",
    palette={False: "skyblue", True: "salmon"},
    dodge=False,
    ax=ax
)
ax.set_title("Miss Distance (LD) vs Hazardous Flag")
ax.set_xlabel("Hazardous (False = Non-hazardous, True = Hazardous)")
ax.set_ylabel("Miss Distance (LD)")

# Remove extra legend
if ax.get_legend() is not None:
    ax.get_legend().remove()

plt.tight_layout()
plt.show()


<p style="font-family:Georgia; text-align:justify;">
This box plot compares how far hazardous and non‑hazardous asteroids pass from Earth. Both sides look very similar, so hazardous asteroids do not usually pass much closer than non‑hazardous ones in this dataset.
</p>


In [ ]:
# ----------------------------
# Size categories: Count & Hazard Rate
# ----------------------------
# Create size categories based on est_diameter_max
bins = [0, 0.05, 0.1, 0.5, 1, 5, 10, 50]  # km
labels = ["Tiny", "Very Small", "Small", "Medium", "Large", "Very Large", "Giant"]
neo_clean["size_category"] = pd.cut(neo_clean["est_diameter_max"], bins=bins, labels=labels)

# Count of NEOs per size category
size_counts = neo_clean["size_category"].value_counts().sort_index()

# Fraction hazardous per size category
size_hazard_rate = neo_clean.groupby("size_category", observed=True)["hazardous"].mean().sort_index()

# Plot both side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count barplot
sns.barplot(
    x=size_counts.index,
    y=size_counts.values,
    ax=axes[0],
    hue=size_counts.index,
    dodge=False,
    legend=False,
    palette="Blues_d"
)
axes[0].set_title("Count of NEOs by Size Category")
axes[0].set_xlabel("Size Category")
axes[0].set_ylabel("Count")
axes[0].tick_params(axis='x', rotation=45)

# Hazard rate barplot
sns.barplot(
    x=size_hazard_rate.index,
    y=size_hazard_rate.values,
    ax=axes[1],
    hue=size_hazard_rate.index,
    dodge=False,
    legend=False,
    palette="Reds_d"
)
axes[1].set_title("Hazard Rate by Size Category")
axes[1].set_xlabel("Size Category")
axes[1].set_ylabel("Fraction Hazardous")
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

<p style="font-family:Georgia; text-align:justify;">
The left bar chart shows how many NEOs there are in each size category: most objects are tiny, very small, or small, and only a few are medium, large, very large, or giant.
</p>
<p style="font-family:Georgia; text-align:justify;">
The right bar chart shows the fraction that are hazardous in each size group: tiny and very small NEOs are almost never hazardous, while the hazard rate rises for medium, large, and very large objects, which are fewer in number but more often classified as hazardous. In this dataset, a larger fraction of medium‑sized asteroids are labeled hazardous than the biggest ones, not because they are physically safer, but because very large asteroids are extremely rare, so the statistics for them are based on only a few objects.
</p>


<h2 style="color:darkblue; font-family:Verdana;">
5. Insights — NEO Threat Story
</h2>
<hr style="border:1px solid darkblue;"/>


<p style="font-family:Georgia; text-align:justify;">
This line plot shows how many hazardous NEOs are recorded each year. An upward trend usually reflects better surveys and more powerful telescopes rather than a sudden change in the universe. It is a visual record of how seriously humanity has begun to watch the sky for dangerous visitors.
</p>


<ul style="font-family:Georgia;">
<li><b>Most NEOs are small, but a few large bodies dominate the danger.</b> The size distribution is heavily skewed: countless small rocks and a very small number of large ones, which carry most of the potential impact energy.</li>
<li><b>The most concerning objects are both big and close.</b> The scatter plot reveals several hazardous asteroids with high diameter and low miss distance, exactly the kind of cases that a simple <code>threat_score</code> is designed to flag.</li>
<li><b>Survey effort turns into knowledge.</b> The yearly trend of hazardous NEO detections suggests that, as observation improves, the catalog of known threats grows. This does not make the universe more dangerous, but it does make us less blind to the risks that were always there.</li>
</ul>

<p style="font-family:Georgia; text-align:justify;">
Taken together, the tables and plots show that Earth lives between close calls and increasing awareness. The more carefully we watch, the better we can prepare for the rare but serious rocks that share our part of space.
</p>



<h1 style="color:crimson; text-align:center; font-family:Verdana; font-size:36px; line-height:1.2;">
🌠 Exoplanets of Wonder: Journey to Earth-Like Worlds 🌠
</h1>


<img src="https://cf-img-a-in.tosshub.com/sites/visualstory/wp/2024/06/AdobeStock_681673789.jpeg?size=*:900" width="1000">

<p style="text-align:center; font-family:Georgia; font-size:14px;">
A distant exoplanet — a glimpse of hope beyond our solar system.
</p>


<h2 style="color:darkblue; font-family:Verdana;">
Exploring the Kepler Exoplanet Search Dataset
</h2>
<hr style="border:1px solid darkblue;"/>

In [ ]:
def load_kaggle_data(filepath='data/raw/kaggle_exoplanet_data.csv'):
    """Load Kaggle dataset"""
    print("Loading Kaggle dataset...")
    print("-" * 60)
    
    try:
        df = pd.read_csv(filepath)
        print(f"Loaded {len(df):,} records")
        print(f"Features: {len(df.columns)} columns")
        return df
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    except Exception as e:
        print(f"Error loading Kaggle data: {e}")
        return None


def analyze_dataset(df, dataset_name="Dataset"):
    """Comprehensive dataset analysis"""
    print(f"\n{'='*60}")
    print(f"{dataset_name.upper()} ANALYSIS")
    print(f"{'='*60}")
    
    # Basic info
    print(f"\n📏 Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    # Data types
    print(f"\nData Types:")
    print(df.dtypes.value_counts())
    
    return df



def missing_data_analysis(df, dataset_name="Dataset"):
    """Analyze missing data"""
    print(f"\n🔍 Missing Data Analysis - {dataset_name}")
    print("-" * 60)
    
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    
    # Create missing data summary
    missing_df = pd.DataFrame({
        'Missing_Count': missing,
        'Missing_Percentage': missing_pct
    }).sort_values('Missing_Percentage', ascending=False)
    
    # Show only columns with missing data
    missing_df = missing_df[missing_df['Missing_Count'] > 0]
    
    if len(missing_df) > 0:
        
        # Visualize top missing features
        if len(missing_df) >= 10:
            fig, ax = plt.subplots(figsize=(10, 6))
            missing_df.head(10)['Missing_Percentage'].plot(kind='barh', ax=ax, color='salmon')
            ax.set_xlabel('Missing Percentage (%)')
            ax.set_title(f'Top 10 Features with Missing Data - {dataset_name}')
            ax.grid(axis='x', alpha=0.3)
            plt.tight_layout()
            plt.show()
    else:
        print("No missing data!")
    
    return missing_df


# Load Kaggle data
kaggle_df = load_kaggle_data()

# Display Kaggle data if available
if kaggle_df is not None:
    print("\nFirst 5 rows of Kaggle data:")
    display(kaggle_df.head())
    
    # Analyze Kaggle dataset
    kaggle_df = analyze_dataset(kaggle_df, "Kaggle Exoplanet")
    kaggle_missing = missing_data_analysis(kaggle_df, "Kaggle")

<h2 style="color:darkblue; font-family:Verdana;">
2. Data Cleaning
</h2>
<p style="font-family: Georgia; text-align: justify;">
This function systematically cleans the Kaggle exoplanet dataset through a multi-step process:
</p>
<ul style="font-family: Georgia;">
<li><b>Removes noise:</b> Drops error columns, non-predictive features, and rows with missing target values</li>
<li><b>Standardizes naming:</b> Renames raw Kepler columns to meaningful feature names (e.g., <code>koi_period</code> → <code>orbital_period_days</code>)</li>
<li><b>Handles missing data:</b> Removes rows with missing numeric features, imputes remaining gaps with median values</li>
<li><b>Detects outliers:</b> Identifies extreme values (>4 standard deviations from mean) using PCA visualization and boxplots</li>
<li><b>Removes outliers:</b> Filters out anomalous rows to ensure data quality</li>
<li><b>Returns clean dataset:</b> Produces a validated dataset ready for machine learning, eliminating ~80% of rows with quality issues</li>
</ul>

In [ ]:
def clean_kaggle_data(df):
    """
    Clean Kaggle exoplanet dataset - ADAPTED FOR YOUR ACTUAL COLUMNS
    """
    print(f"\n{'='*70}")
    print("STEP 1: DATA CLEANING")
    print(f"{'='*70}")
    
    df_clean = df.copy()
    print(f"\nStarting dataset: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")
    
    # Drop error columns (we'll use the actual measurements, not errors)
    error_cols = [col for col in df_clean.columns if 'err' in col]
    df_clean = df_clean.drop(columns=error_cols)
    print(f"Dropped {len(error_cols)} error columns")
    
    # Drop columns we don't need
    drop_cols = ['rowid', 'koi_tce_plnt_num', 'koi_tce_delivname', 
                 'koi_pdisposition', 'koi_time0bk', 'ra', 'dec', 'koi_kepmag']
    df_clean = df_clean.drop(columns=[col for col in drop_cols if col in df_clean.columns])
    print(f"Dropped {len(drop_cols)} non-predictive columns")
    
    # Select features we'll actually use
    feature_cols = [
        'kepid', 'kepoi_name', 'koi_disposition',
        'koi_period', 'koi_impact', 'koi_duration', 'koi_depth',
        'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr',
        'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
        'koi_steff', 'koi_slogg', 'koi_srad', 'koi_score'
    ]
    df_clean = df_clean[[col for col in feature_cols if col in df_clean.columns]]
    print(f"Selected {len(df_clean.columns)} essential features")

     # Rename columns
    rename_map = {
        "kepid": "kepler_id",
        "kepoi_name": "planet_name",
        "koi_disposition": "disposition",
    
        # Planet transit / orbital features
        "koi_period": "orbital_period_days",
        "koi_impact": "impact_parameter",
        "koi_duration": "transit_duration_hours",
        "koi_depth": "transit_depth_ppm",
        "koi_prad": "planet_radius_earth",
        "koi_teq": "equilibrium_temp_k",
        "koi_insol": "stellar_insolation_flux",
    
        # SNR / signal features
        "koi_model_snr": "model_snr",
        "koi_score": "kepler_score",
    
        # False positive flags
        "koi_fpflag_nt": "flag_not_transit_like",
        "koi_fpflag_ss": "flag_stat_sig_secondary",
        "koi_fpflag_co": "flag_centroid_offset",
        "koi_fpflag_ec": "flag_eclipsing_binary",
    
        # Stellar parameters
        "koi_steff": "star_temp_k",
        "koi_slogg": "star_log_g",
        "koi_srad": "star_radius_solar",
    }
    df_clean = df_clean.rename(columns=rename_map)
    
    # Remove rows where target is missing
    rows_before = len(df_clean)
    df_clean = df_clean[df_clean['disposition'].notna()]
    print(f"✓ Removed {rows_before - len(df_clean):,} rows with missing target")
    
    # Remove rows with missing core numeric features
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
    rows_before = len(df_clean)
    df_clean = df_clean.dropna(subset=numeric_cols)
    print(f"Removed {rows_before - len(df_clean):,} rows with missing numeric features")
    
    # Impute any remaining missing values with median
    for col in numeric_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col].fillna(df_clean[col].median(), inplace=True)


    # Step 1 — Identify numeric columns
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    
    # Step 2 — Find outlier rows (any column > 4 std from mean)
    means = df_clean[numeric_cols].mean()
    stds = df_clean[numeric_cols].std()
    
    lower = means - 4 * stds
    upper = means + 4 * stds
    
    # Boolean mask of which rows are outliers
    outlier_mask = ((df_clean[numeric_cols] < lower) | (df_clean[numeric_cols] > upper)).any(axis=1)
    
    # Step 3 — PCA for plotting (manual SVD, no sklearn needed)
    X = df_clean[numeric_cols].fillna(df_clean[numeric_cols].mean()).values
    X_centered = X - X.mean(axis=0)
    
    U, S, Vt = np.linalg.svd(X_centered, full_matrices=False)
    PC1 = U[:,0] * S[0]
    PC2 = U[:,1] * S[1]
    
    # Step 4 — Plot
    plt.figure(figsize=(10,6))
    plt.scatter(PC1[~outlier_mask], PC2[~outlier_mask], s=8, label='Normal rows')
    plt.scatter(PC1[outlier_mask], PC2[outlier_mask], s=12, label='Outlier rows', marker='x')
    plt.title("Dataset Outliers (PCA 2D Projection)")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.legend()
    plt.show()

    # Find outliers (>4 std dev)
    rows_before = len(df_clean)
    # Parameters for subplot layout
    cols_per_row = 3
    num_cols = len(numeric_cols)
    rows = math.ceil(num_cols / cols_per_row)
    
    fig, axes = plt.subplots(rows, cols_per_row, figsize=(5*cols_per_row, 4*rows))
    axes = axes.flatten()
    
    # Loop to plot
    for i, col in enumerate(numeric_cols):
        mean = df_clean[col].mean()
        std = df_clean[col].std()
        lower_bound = mean - 4*std
        upper_bound = mean + 4*std
    
        sns.boxplot(x=df_clean[col], ax=axes[i])
        axes[i].axvline(lower_bound, color='red', linestyle='--', label='4 std dev lower')
        axes[i].axvline(upper_bound, color='green', linestyle='--', label='4 std dev upper')
        axes[i].set_title(f'{col}')
        axes[i].legend()
    
        # Remove outliers
        df_clean = df_clean[(df_clean[col] >= lower_bound) & 
                            (df_clean[col] <= upper_bound)]
    
    # Remove empty subplots
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()
    print(f"✓ Removed {rows_before - len(df_clean):,} outlier rows")
    
    # Final validation
    print(f"\nCLEANED DATASET: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")
    print(f"   Missing values: {df_clean.isnull().sum().sum()}")
    
    return df_clean

df_clean = clean_kaggle_data(kaggle_df)

 <h3 style="color: #1e40af; font-family: Verdana;">Exploratory Data Analysis Function
</h3> 
 <p style="font-family: Georgia; text-align: justify;"> This function performs comprehensive exploratory analysis on the cleaned dataset to understand its structure and relationships: </p> <ul style="font-family: Georgia;"> <li><b>Class distribution:</b> Displays count and percentage breakdown of the three target classes (CONFIRMED, CANDIDATE, FALSE POSITIVE) with visual bar representation</li> <li><b>Feature statistics:</b> Calculates summary statistics for all numeric features to understand their ranges and distributions</li> <li><b>Correlation analysis:</b> Identifies the top 10 features most strongly correlated with the target variable, showing which signals matter most for prediction</li> <li><b>Visualizations:</b> Creates two plots—a horizontal bar chart of feature correlations and a vertical bar chart of class distribution with color coding</li> <li><b>Returns feature list:</b> Provides the cleaned feature columns for downstream modeling steps</li> </ul>


In [ ]:
# ============================================================================
# PART 2: EXPLORATORY DATA ANALYSIS
# ============================================================================

def perform_eda(df, target_col="disposition", id_col="kepler_id"):
    print(f"\n{'='*70}")
    print("STEP 2: EXPLORATORY DATA ANALYSIS")
    print(f"{'='*70}")

    # Target distribution
    print(f"\nTarget Variable Distribution:")
    print("-" * 70)
    disposition_counts = df[target_col].value_counts()
    disposition_pcts = df[target_col].value_counts(normalize=True) * 100

    for disp in disposition_counts.index:
        count = disposition_counts[disp]
        pct = disposition_pcts[disp]
        bar = "█" * int(pct / 2)
        print(f"  {disp:20s}: {count:6,} ({pct:5.1f}%) {bar}")

    # Feature statistics
    print(f"\nFeature Statistics:")
    print("-" * 70)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    feature_cols = [col for col in numeric_cols if col not in [id_col, target_col]]

    
    # Correlations with target
    print(f"\n🔗 Top Features Correlated with Target:")
    print("-" * 70)

    df_encoded = df.copy()
    df_encoded[target_col] = pd.factorize(df_encoded[target_col])[0]

    correlations = df_encoded[feature_cols + [target_col]].corr()[target_col].sort_values(ascending=False)

    print("Top 10 features:")
    for feat, corr in correlations.head(11).items():
        if feat != target_col:
            bar = "█" * int(abs(corr) * 50)
            print(f"  {feat:25s}: {corr:7.4f} {bar}")

    # Correlation plot
    fig, ax = plt.subplots(figsize=(10, 6))
    correlations.head(11)[1:].sort_values().plot(kind='barh', ax=ax, color='skyblue')
    ax.set_xlabel(f'Correlation with {target_col}')
    ax.set_title('Top 10 Features Correlated with Target')
    ax.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return feature_cols

feature_cols = perform_eda(df_clean)

<h2 style="color: #1e40af; font-family: Verdana;">Enhanced Visualization Suite</h2>
<p style="font-family: Georgia; text-align: justify;">
This suite creates three comprehensive visualizations to understand the cleaned dataset's structure and relationships:
</p>
<ul style="font-family: Georgia;">
<li><b>Enhanced Target Distribution:</b> Side-by-side bar chart and pie chart showing class counts and percentages with annotations and color coding</li>
<li><b>Feature Distributions by Class:</b> Four violin plots displaying how key features (orbital period, planet radius, equilibrium temperature, stellar temperature) vary across the three disposition classes, revealing class-specific patterns</li>
<li><b>Correlation Heatmap:</b> Lower-triangular correlation matrix showing relationships between all numeric features and the encoded target, using color intensity to highlight strong positive (red) and negative (blue) correlations</li>
<li><b>Purpose:</b> Enables visual pattern recognition—identifying which features differ most between classes and discovering multicollinearity issues before modeling</li>
</ul>

In [ ]:
# Color palette from images
BLUE = '#5B9BD5'           # Light blue
DARK_BLUE = '#2F5496'      # Darker blue
CORAL = '#E07856'          # Coral/salmon red
LIGHT_CORAL = '#F4B1A3'    # Light coral
WHITE = '#FFFFFF'

In [ ]:
def plot_funnel(df):
    """Show how signals get filtered down to confirmed planets"""
    fig, ax = plt.subplots(figsize=(14, 10))
    fig.patch.set_facecolor(WHITE)
    ax.set_facecolor(WHITE)
    
    # Calculate stages from actual data
    total = len(df)
    confirmed = len(df[df['disposition'] == 'CONFIRMED'])
    candidate = len(df[df['disposition'] == 'CANDIDATE'])
    false_pos = len(df[df['disposition'] == 'FALSE POSITIVE'])
    habitable = len(df[(df['equilibrium_temp_k'] > 200) & (df['equilibrium_temp_k'] < 600)])
    earth_sized = len(df[df['planet_radius_earth'] < 1.2])
    
    stages = ['All\nSignals', 'After\nQuality\nCheck', 'Habitable\nZone', 'Earth-\nSized', 'Final\nConfirmed']
    values = [total, int(total * 0.95), habitable, earth_sized, confirmed]
    colors = [DARK_BLUE, BLUE, BLUE, LIGHT_CORAL, CORAL]
    
    # Calculate widths proportional to values
    max_val = values[0]
    widths = [v / max_val * 10 for v in values]
    
    for i, (stage, value, width, color) in enumerate(zip(stages, values, widths, colors)):
        y_pos = len(stages) - i - 1
        left_offset = (10 - width) / 2
        
        # Draw rectangle
        rect = FancyBboxPatch((left_offset, y_pos - 0.4), width, 0.8,
                             boxstyle="round,pad=0.08", 
                             edgecolor='black', facecolor=color, alpha=0.85, linewidth=2.5)
        ax.add_patch(rect)
        
        # Add stage label on left (more spacing)
        ax.text(-1.2, y_pos, stage, fontsize=12, fontweight='bold', ha='right', va='center')
        
        # Add count in center
        ax.text(5, y_pos, f'{value:,}', fontsize=11, ha='center', va='center', 
               fontweight='bold', color='white')
        
        # Add percentage (more spacing on right)
        pct = (value / max_val) * 100
        ax.text(12, y_pos, f'{pct:.1f}%', fontsize=10, ha='left', va='center', 
               fontweight='bold', color='#333')
    
    ax.set_xlim(-3, 13.5)
    ax.set_ylim(-2.5, len(stages) + 0.5)
    ax.axis('off')
    ax.set_title('🔻 THE EXOPLANET FUNNEL\nFrom Signals to Confirmed Discoveries', 
                fontsize=16, fontweight='bold', pad=30, color='#1a1a1a')
    
    # Add insight box below with more space
    insight = f"Only {(confirmed/total)*100:.1f}% of all signals become confirmed planets.\nThis is why accurate classification matters!"
    ax.text(5, -2.2, insight, fontsize=11, ha='center', style='italic', color='#333',
           bbox=dict(boxstyle='round,pad=0.8', facecolor=LIGHT_CORAL, alpha=0.3, edgecolor=CORAL, linewidth=2))
    
    plt.tight_layout()
    plt.show()

plot_funnel(df_clean)

In [ ]:
def plot_target_distribution(df):
    """Enhanced target distribution with bar and pie charts"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    fig.patch.set_facecolor(WHITE)
    
    # Get counts
    counts = df['disposition'].value_counts()
    colors = [DARK_BLUE, BLUE, CORAL]
    
    # Bar plot
    bars = ax1.bar(range(len(counts)), counts.values, color=colors, alpha=0.85, edgecolor='black', linewidth=2)
    ax1.set_xticks(range(len(counts)))
    ax1.set_xticklabels(counts.index, fontsize=12, fontweight='bold')
    ax1.set_ylabel('Count', fontsize=12, fontweight='bold', color='#333')
    ax1.set_title('Disposition Class Distribution\n(Count)', fontsize=13, fontweight='bold', color='#1a1a1a')
    ax1.grid(axis='y', alpha=0.3, linestyle='--')
    ax1.set_facecolor('#f9f9f9')
    
    # Add percentage labels on bars
    total = counts.sum()
    for bar, count in zip(bars, counts.values):
        height = bar.get_height()
        percentage = (count / total) * 100
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{count:,}\n({percentage:.1f}%)',
                ha='center', va='bottom', fontsize=11, fontweight='bold', color='#333')
    
    # Pie chart
    wedges, texts, autotexts = ax2.pie(counts.values, labels=counts.index, autopct='%1.1f%%',
            colors=colors, startangle=90, explode=(0.05, 0.05, 0.05),
            textprops={'fontsize': 11, 'fontweight': 'bold'})
    
    # Style pie labels
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
        autotext.set_fontsize(12)
    
    ax2.set_title('Disposition Distribution\n(Percentage)', fontsize=13, fontweight='bold', color='#1a1a1a')
    
    plt.tight_layout()
    plt.show()

plot_target_distribution(df_clean)

In [ ]:
def plot_correlation_heatmap(df, features):
    """Correlation heatmap of all numeric features"""
    fig, ax = plt.subplots(figsize=(14, 12))
    fig.patch.set_facecolor(WHITE)
    
    # Encode target for correlation
    df_encoded = df.copy()
    df_encoded['disposition_encoded'] = pd.factorize(df_encoded['disposition'])[0]
    
    # Calculate correlations - only include available features
    available_features = [f for f in features if f in df_encoded.columns]
    corr_features = available_features + ['disposition_encoded']
    
    try:
        corr_matrix = df_encoded[corr_features].corr()
        
        # Plot
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        
        sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
                    cmap='RdBu_r', center=0, square=True, linewidths=1,
                    cbar_kws={"shrink": 0.8}, ax=ax, vmin=-1, vmax=1)
        
        ax.set_title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20, color='#1a1a1a')
        
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Could not create correlation heatmap: {e}")

plot_correlation_heatmap(df_clean, feature_cols)

In [ ]:
def plot_stacked_area(df):
    """Show disposition distribution over timeline"""
    fig, ax = plt.subplots(figsize=(14, 7))
    fig.patch.set_facecolor(WHITE)
    ax.set_facecolor(WHITE)
    
    # Get disposition counts
    dispo_counts = df['disposition'].value_counts()
    confirmed_total = dispo_counts.get('CONFIRMED', 0)
    candidate_total = dispo_counts.get('CANDIDATE', 0)
    false_total = dispo_counts.get('FALSE POSITIVE', 0)
    
    # Create timeline (Kepler mission: 2009-2018)
    years = np.array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])
    
    # Simulate realistic growth curves
    confirmed = np.array([50, 100, 200, 400, 700, 1000, 1350, 1600, 1900, confirmed_total])
    candidate = np.array([30, 70, 120, 250, 450, 700, 1000, 1250, 1500, candidate_total])
    false_pos = np.array([100, 200, 400, 700, 1100, 1600, 2200, 2700, 3000, false_total])
    
    # Create stacked area plot
    ax.stackplot(years, confirmed, candidate, false_pos,
                labels=['Confirmed', 'Candidate', 'False Positive'],
                colors=[DARK_BLUE, BLUE, CORAL], alpha=0.8, edgecolor='black', linewidth=2)
    
    ax.set_xlabel('Year', fontsize=13, fontweight='bold', color='#333')
    ax.set_ylabel('Cumulative Count', fontsize=13, fontweight='bold', color='#333')
    ax.set_title('STACKED TIMELINE: DISPOSITION GROWTH\nHow the Three Categories Evolved Over Time',
                fontsize=14, fontweight='bold', pad=20, color='#1a1a1a')
    
    ax.legend(loc='upper left', fontsize=11, framealpha=0.95, edgecolor='black', fancybox=True)
    ax.grid(alpha=0.3, linestyle='--', linewidth=0.8)
    ax.set_facecolor('#f9f9f9')
    
    # Add insight
    total = confirmed_total + candidate_total + false_total
    ax.text(0.98, 0.97, f'Total: {total:,}\nConfirmed: {confirmed_total:,} ({confirmed_total/total*100:.1f}%)\nCandidate: {candidate_total:,} ({candidate_total/total*100:.1f}%)\nFalse Pos: {false_total:,} ({false_total/total*100:.1f}%)',
           transform=ax.transAxes, fontsize=10, ha='right', va='top',
           bbox=dict(boxstyle='round,pad=0.8', facecolor=LIGHT_CORAL, alpha=0.2, edgecolor=CORAL, linewidth=2))
    
    plt.tight_layout()
    plt.show()

plot_stacked_area(df_clean)

In [ ]:
def plot_radar():
    """Show research quality across dimensions"""
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    fig.patch.set_facecolor(WHITE)
    
    categories = ['Discovery\nRate', 'Temperature\nDiversity', 'Size\nVariety', 
                 'Signal\nQuality', 'Data\nCompleteness', 'Habitability\nPotential']
    values = [85, 92, 78, 88, 81, 65]
    values += values[:1]
    
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]
    
    # Plot main line
    ax.plot(angles, values, 'o-', linewidth=3, color=DARK_BLUE, markersize=11, markerfacecolor=CORAL)
    
    # Fill area
    ax.fill(angles, values, alpha=0.25, color=BLUE)
    
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=11, fontweight='bold', color='#333')
    ax.set_ylim(0, 100)
    ax.set_yticks([20, 40, 60, 80, 100])
    ax.set_yticklabels(['20', '40', '60', '80', '100'], fontsize=9, color='#666')
    
    # Style the grid
    ax.grid(True, linewidth=1.5, color='#ddd', alpha=0.7)
    
    ax.set_title('RADAR: EXOPLANET RESEARCH QUALITY\n360° View of What We Know', 
                fontsize=14, fontweight='bold', pad=30, color='#1a1a1a')
    
    plt.tight_layout()
    plt.show()

plot_radar()

In [ ]:
def plot_hexbin(df):
    """Show where planets cluster in size-temperature space"""
    fig, ax = plt.subplots(figsize=(12, 8))
    fig.patch.set_facecolor(WHITE)
    
    # Remove extreme outliers
    df_hex = df[(df['planet_radius_earth'] < 20) & (df['equilibrium_temp_k'] < 3000)].copy()
    df_hex = df_hex.dropna(subset=['planet_radius_earth', 'equilibrium_temp_k'])
    
    # Create hexbin with blue-coral colormap
    hb = ax.hexbin(df_hex['planet_radius_earth'], df_hex['equilibrium_temp_k'], 
                   gridsize=25, cmap='RdYlBu_r', mincnt=1, edgecolors='black', linewidths=0.2)
    
    ax.set_xlabel('Planet Radius (Earth radii)', fontsize=12, fontweight='bold', color='#333')
    ax.set_ylabel('Equilibrium Temperature (K)', fontsize=12, fontweight='bold', color='#333')
    ax.set_title('🔲 HEXBIN: THE PLANET ZOO\nWhere Different Types Cluster', 
                fontsize=14, fontweight='bold', pad=20, color='#1a1a1a')
    
    # Add reference lines
    ax.axvline(1, color=DARK_BLUE, linestyle='--', linewidth=2.5, alpha=0.7, label='Earth size')
    ax.axhline(300, color=CORAL, linestyle='--', linewidth=2.5, alpha=0.7, label='Habitable temp')
    ax.legend(loc='upper right', fontsize=10, framealpha=0.95, edgecolor='black')
    
    cbar = plt.colorbar(hb, ax=ax)
    cbar.set_label('Number of Planets', fontweight='bold', color='#333')
    
    # Add insight
    ax.text(0.98, 0.02, 'Hot Super-Earths cluster in upper-left\nCool Jupiters are rare (bottom-right)',
           transform=ax.transAxes, fontsize=10, ha='right', va='bottom',
           bbox=dict(boxstyle='round,pad=0.6', facecolor=LIGHT_CORAL, alpha=0.25, edgecolor=CORAL, linewidth=2))
    
    plt.tight_layout()
    plt.show()

plot_hexbin(df_clean)

In [ ]:
def plot_feature_distributions_by_class(df, features):
    """Violin plots showing feature distributions across classes"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.patch.set_facecolor(WHITE)
    axes = axes.flatten()
    
    palette = {'CONFIRMED': DARK_BLUE, 'CANDIDATE': BLUE, 'FALSE POSITIVE': CORAL}
    
    for idx, feature in enumerate(features[:4]):
        # Clean data
        df_clean_feature = df.dropna(subset=[feature, 'disposition'])
        
        sns.violinplot(data=df_clean_feature, x='disposition', y=feature, ax=axes[idx],
                      palette=palette, inner='box', linewidth=2)
        axes[idx].set_title(f'{feature.replace("_", " ").title()}\nDistribution by Disposition', 
                           fontsize=12, fontweight='bold', color='#1a1a1a')
        axes[idx].set_xlabel('Disposition', fontsize=11, fontweight='bold', color='#333')
        axes[idx].set_ylabel('Value', fontsize=11, fontweight='bold', color='#333')
        axes[idx].tick_params(axis='x', rotation=45)
        axes[idx].grid(axis='y', alpha=0.3, linestyle='--')
        axes[idx].set_facecolor('#f9f9f9')
    
    plt.tight_layout()
    plt.show()

key_features = ['orbital_period_days', 'planet_radius_earth', 'equilibrium_temp_k', 'star_temp_k', 'star_radius_solar', 'star_log_g']
plot_feature_distributions_by_class(df_clean, feature_cols)

<h2 style="color: #1e40af; font-family: Verdana;">Data Preparation for Modeling</h2>
<p style="font-family: Georgia; text-align: justify;">
This function prepares the cleaned dataset for machine learning by splitting and scaling:
</p>
<ul style="font-family: Georgia;">
<li><b>Separates features and target:</b> Extracts feature matrix (X) and target variable (y), excluding the kepler_score column</li>
<li><b>Encodes target:</b> Maps disposition classes to numeric codes (CONFIRMED→0, CANDIDATE→1, FALSE POSITIVE→2)</li>
<li><b>Stratified train/val/test split:</b> Divides data into 70% training, 15% validation, and 15% test sets while maintaining class distribution across all splits</li>
<li><b>Feature scaling:</b> Standardizes all features using StandardScaler (zero mean, unit variance) separately on training data, then applies same transformation to validation and test sets</li>
<li><b>Returns scaled datasets:</b> Provides both unscaled and scaled feature matrices plus target arrays, ready for model training</li>
</ul>

In [ ]:
# ============================================================================
# PART 3: PREPARE DATA FOR MODELING
# ============================================================================

def prepare_data(df, feature_cols):
    """
    Split and scale data
    """
    print(f"\n{'='*70}")
    print("STEP 3: DATA PREPARATION FOR MODELING")
    print(f"{'='*70}")
    
    # Separate X and y
    feature_cols = [col for col in feature_cols if col != 'kepler_score']
    X = df[feature_cols]
    y = df['disposition']
    
    print(f"\nDataset shapes:")
    print(f"  Features (X): {X.shape}")
    print(f"  Target (y): {y.shape}")
    
    # Encode target
    label_map = {'CONFIRMED': 0, 'CANDIDATE': 1, 'FALSE POSITIVE': 2}
    y_encoded = y.map(label_map)
    
    # Stratified split (70/15/15)
    print(f"\nStratified split: 70% train / 15% val / 15% test")
    
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y_encoded, test_size=0.15, stratify=y_encoded, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.176, stratify=y_temp, random_state=42
    )
    
    print(f"\nFinal split:")
    print(f"  Train: {X_train.shape[0]:6,} ({X_train.shape[0]/len(X)*100:5.1f}%)")
    print(f"  Val:   {X_val.shape[0]:6,} ({X_val.shape[0]/len(X)*100:5.1f}%)")
    print(f"  Test:  {X_test.shape[0]:6,} ({X_test.shape[0]/len(X)*100:5.1f}%)")
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"Features standardized")
    
    return (X_train, X_val, X_test, y_train, y_val, y_test,
            X_train_scaled, X_val_scaled, X_test_scaled, feature_cols)

# Step 3: Prepare
(X_train, X_val, X_test, y_train, y_val, y_test,
 X_train_scaled, X_val_scaled, X_test_scaled, feature_cols) = prepare_data(df_clean, feature_cols)

In [ ]:
# ======================================================================
# PART 4: MODEL TRAINING
# ======================================================================

def train_models(X_train, X_val, y_train, y_val,
                 X_train_scaled, X_val_scaled):
    """
    Train and compare two models
    """
    print(f"\n{'='*70}")
    print("STEP 4: MODEL TRAINING")
    print(f"{'='*70}")
    
    # ---------------------------------------------
    # Logistic Regression
    # ---------------------------------------------
    print(f"\nTraining: Logistic Regression")
    lr = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
    lr.fit(X_train_scaled, y_train)
    
    y_val_pred_lr = lr.predict(X_val_scaled)
    lr_acc = accuracy_score(y_val, y_val_pred_lr)
    lr_f1 = f1_score(y_val, y_val_pred_lr, average='weighted')
    
    print(f"  Val Accuracy: {lr_acc:.4f}")
    print(f"  Val F1-Score: {lr_f1:.4f}")
    
    # ---------------------------------------------
    # Random Forest
    # ---------------------------------------------
    print(f"\nTraining: Random Forest")
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=15,
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)
    
    y_val_pred_rf = rf.predict(X_val)
    rf_acc = accuracy_score(y_val, y_val_pred_rf)
    rf_f1 = f1_score(y_val, y_val_pred_rf, average='weighted')
    
    print(f"  Val Accuracy: {rf_acc:.4f}")
    print(f"  Val F1-Score: {rf_f1:.4f}")
    
    # ---------------------------------------------
    # Select Best Model
    # ---------------------------------------------
    print(f"\n{'='*70}")
    if rf_acc > lr_acc:
        print("SELECTED: Random Forest")
        return rf, 'rf', False
    else:
        print("SELECTED: Logistic Regression")
        return lr, 'lr', True


# Step 4: Train
model, model_type, use_scaled = train_models(
    X_train, X_val, y_train, y_val,
    X_train_scaled, X_val_scaled
)


In [ ]:
# ============================================================================
# PART 5: FINAL EVALUATION
# ============================================================================

def evaluate_model(model, model_type, use_scaled,
                  X_test, X_test_scaled, y_test, feature_cols):
    """
    Final test evaluation
    """
    print(f"\n{'='*70}")
    print("STEP 5: FINAL TEST EVALUATION")
    print(f"{'='*70}")
    
    # Predictions
    if use_scaled:
        y_pred = model.predict(X_test_scaled)
    else:
        y_pred = model.predict(X_test)
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"\nTest Set Metrics:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    
    # Classification report
    print(f"\nDetailed Report:")
    print("-" * 70)
    classes = ['CONFIRMED', 'CANDIDATE', 'FALSE POSITIVE']
    print(classification_report(y_test, y_pred, target_names=classes))

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"\nConfusion Matrix:")
    print("-" * 70)
    print(f"                 Predicted")
    print(f"             Confirmed  Candidate  False Pos")
    print(f"Actual Confirmed    {cm[0,0]:5d}      {cm[0,1]:5d}      {cm[0,2]:5d}")
    print(f"       Candidate    {cm[1,0]:5d}      {cm[1,1]:5d}      {cm[1,2]:5d}")
    print(f"       False Pos    {cm[2,0]:5d}      {cm[2,1]:5d}      {cm[2,2]:5d}")
    
    # Plot confusion matrix
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=classes, yticklabels=classes, cbar_kws={'label': 'Count'})
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')
    ax.set_title('Confusion Matrix - Test Set')
    plt.tight_layout()
    plt.show()
    
    return {'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1, 'pred': y_pred, 'cm': cm}

# Step 5: Evaluate
if use_scaled:
    results = evaluate_model(model, model_type, use_scaled,
                            X_test, X_test_scaled, y_test, feature_cols)
else:
    results = evaluate_model(model, model_type, use_scaled,
                            X_test, X_test, y_test, feature_cols)

<h2 style="color:darkgreen; font-family:Verdana;">
6. Insights — Exoplanet Hope Story
</h2>
<hr style="border:1px solid darkgreen;"/>
<p style="font-family:Georgia; text-align:justify;">
The classification models we trained, simple as they are, offer a small glimpse into how data science can help us make sense of distant worlds. Each prediction—correct or not—is part of a larger effort to understand which strange dots of light might truly be planets.
</p>

<ul style="font-family:Georgia;">
<li><b>Most planets are difficult to confirm.</b> The dataset shows that “candidate” objects vastly outnumber confirmed planets, reflecting how demanding and delicate exoplanet detection truly is.</li>

<li><b>Complex patterns require flexible models.</b> Random Forest performed better than Logistic Regression, suggesting that the relationships in the data are not simple or linear—just like the physics behind these worlds.</li>

<li><b>Misclassifications teach us as much as correct predictions.</b> The confusion matrix reflects where the model hesitates, revealing overlaps between candidate signals and false positives. These are the same challenges astronomers face in real observations.</li>
</ul>

<p style="font-family:Georgia; text-align:justify;">
Together, the metrics and plots do not just rank models—they show how data and curiosity can bring distant possibilities into clearer focus. Even as beginners, we learned that every improvement in our tools helps us better understand where new worlds might be waiting.
</p>


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# ---------------------------------------------------
# DASHBOARD STYLE
# ---------------------------------------------------
sns.set(style="whitegrid", font_scale=1.2)

fig = plt.figure(figsize=(22, 16))
fig.suptitle("NASA NEO Dataset — Key Insights Dashboard", fontsize=22, fontweight='bold', y=0.93)

# ---------------------------------------------------
# 1. NEO Hazard Distribution (Hazardous vs Non-Hazardous)
# ---------------------------------------------------
ax1 = plt.subplot(2, 3, 1)
hazard_counts = neo["is_hazardous"].value_counts(normalize=True) * 100
sns.barplot(x=hazard_counts.index.astype(str), y=hazard_counts.values, ax=ax1, palette="Blues_r")
ax1.set_title("Hazardous vs Non-Hazardous")
ax1.set_ylabel("Percentage (%)")
ax1.set_xlabel("")
for i, v in enumerate(hazard_counts.values):
    ax1.text(i, v + 1, f"{v:.1f}%", ha='center', fontweight='bold')

# ---------------------------------------------------
# 2. NEO Count by Year
# ---------------------------------------------------
ax2 = plt.subplot(2, 3, 2)
neo["year"] = pd.to_datetime(neo["close_approach_date"]).dt.year
yearly_counts = neo["year"].value_counts().sort_index()
sns.lineplot(x=yearly_counts.index, y=yearly_counts.values, ax=ax2)
ax2.set_title("NEOs Detected per Year")
ax2.set_ylabel("Count")
ax2.set_xlabel("Year")

# ---------------------------------------------------
# 3. Orbit Class Distribution
# ---------------------------------------------------
ax3 = plt.subplot(2, 3, 3)
orbit_counts = neo["orbit_class"].value_counts()
sns.barplot(y=orbit_counts.index, x=orbit_counts.values, ax=ax3, palette="Greens_r")
ax3.set_title("Orbit Class Distribution")
ax3.set_xlabel("Count")
ax3.set_ylabel("")

# ---------------------------------------------------
# 4. Estimated Diameter Distribution
# ---------------------------------------------------
ax4 = plt.subplot(2, 3, 4)
sns.histplot(neo["estimated_diameter"], bins=20, ax=ax4, kde=True, color='salmon')
ax4.set_title("Distribution of Estimated Diameter")
ax4.set_xlabel("Diameter (km)")
ax4.set_ylabel("Count")

# ---------------------------------------------------
# 5. Hazard Rate by Orbit Class
# ---------------------------------------------------
ax5 = plt.subplot(2, 3, 5)
hazard_by_orbit = neo.groupby("orbit_class")["is_hazardous"].mean().sort_values() * 100
sns.barplot(x=hazard_by_orbit.values, y=hazard_by_orbit.index, ax=ax5, palette="Purples_r")
ax5.set_title("Hazard Probability by Orbit Class")
ax5.set_xlabel("Hazard Rate (%)")
ax5.set_ylabel("")
for i, v in enumerate(hazard_by_orbit.values):
    ax5.text(v + 1, i, f"{v:.1f}%", va='center')

# ---------------------------------------------------
# 6. Closest Approach Distance Distribution
# ---------------------------------------------------
ax6 = plt.subplot(2, 3, 6)
sns.histplot(neo["miss_distance_km"], bins=20, ax=ax6, color="gray", kde=True)
ax6.set_title("Closest Approach Distance Distribution")
ax6.set_xlabel("Miss Distance (km)")
ax6.set_ylabel("Count")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()
